In [111]:
import os
import shutil


def recreate_folder():
    # Path of the folder to be removed
    folder_path = 'data/test'
    try:
        # Remove the folder and its subfolders
        shutil.rmtree(folder_path)
    except:
        pass
    # Create the new folder and its subfolders
    os.makedirs(os.path.join(folder_path, 'fea_sample'))


recreate_folder()


In [112]:
from tabulate import tabulate

def show_graph_info(graph):
    # Compute the number of nodes and edges of the graph
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    # Compute in-degree and out-degree of each node
    in_degrees = dict(graph.in_degree())
    out_degrees = dict(graph.out_degree())
    # Compute average in-degree and out-degree
    avg_in_degree = sum(in_degrees.values()) / len(in_degrees)
    avg_out_degree = sum(out_degrees.values()) / len(out_degrees)
   # Create a table with the graph information
    table = [
        ["Number of nodes", num_nodes],
        ["Number of edges", num_edges],
        ["Average in-degree", f"{avg_in_degree:.2f}"],
        ["Average out-degree", f"{avg_out_degree:.2f}"]
    ]
    # Print the table
    print(tabulate(table, headers=["Metrices", "Value"]))


In [113]:
import pandas as pd
import networkx as nx

df_tmp = pd.read_csv('data/new_train_data.csv')
df_train = df_tmp[df_tmp['label'] == 1]
df_train = df_train.drop(['label'], axis=1)
df_train.to_csv(
    'data/test/test_woheader.csv', header=False, index=False)
# storing the list of edges in a varible
G = nx.read_edgelist('data/test/test_woheader.csv', delimiter=',',
                     create_using=nx.DiGraph(), nodetype=int)

# Printing the information of graph
show_graph_info(G)


Metrices               Value
------------------  --------
Number of nodes     10230
Number of edges     12000
Average in-degree       1.17
Average out-degree      1.17


In [114]:
def compute_shortest_path_length(a, b, graph):
    p = -1
    try:
        if graph.has_edge(a, b):
            graph.remove_edge(a, b)
            p = nx.shortest_path_length(graph, source=a, target=b)
            graph.add_edge(a, b)
        else:
            p = nx.shortest_path_length(graph, source=a, target=b)
        return p
    except:
        return -1


In [115]:
def belongs_to_same_wcc(a, b, graph, wcc):
    index = []
    if graph.has_edge(b, a):
        return 1
    if graph.has_edge(a, b):
        for i in wcc:
            if a in i:
                index = i
                break
        if (b in index):
            graph.remove_edge(a, b)
            if compute_shortest_path_length(a, b, graph) == -1:
                graph.add_edge(a, b)
                return 0
            else:
                graph.add_edge(a, b)
                return 1
        else:
            return 0
    else:
        for i in wcc:
            if a in i:
                index = i
                break
        if (b in index):
            return 1
        else:
            return 0


In [116]:
import numpy as np

wcc = list(nx.weakly_connected_components(G))

df_test = pd.read_csv('data/new_test_data.csv')
df_test['same_comp'] = df_test.apply(
    lambda row: belongs_to_same_wcc(row['node1'], row['node2'], G, wcc), axis=1)

np.bincount(df_test['same_comp'])

array([4077, 1923])

In [117]:
target_csv = 'data/sample_submit.csv'

# Load the CSV file into a Pandas DataFrame
df_target = pd.read_csv(target_csv)

# Replace the values in the 'ans' column with the values from the Python list
df_target['ans'] = df_test['same_comp']

# Save the updated DataFrame back to a CSV file, overwriting the original file
df_target.to_csv(target_csv, index=False)
